# Reliability anaysis


## Packages and Helpers

In [1]:
# packages
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from utils import load_dataframes
from scipy.stats import pearsonr
import random
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import pingouin as pg
import seaborn as sns

## Read Processed Data

In [2]:
# load overall df
random_all_data = pd.read_csv("simulation_random/items_per_LLM_random_simulation.csv")
semi_random_all_data = pd.read_csv("simulation_semi_random/items_per_LLM_semi_random_simulation.csv")


## Normalize item scores where tasks have different scales on different items
- AUDIT, FTND, GABS, PG

In [3]:
def normalize_per_scale(data):
    scales_to_normalize = ["AUDIT scale", "FTND scale", "GABS scale", "PG scale"]

    data = data.copy()

    # Loop over each experiment you want to normalize
    for scale in scales_to_normalize:
        mask = data["experiment"] == scale

        if mask.sum() == 0:
            continue  # skip if scale not present

        # Normalize score
        data.loc[mask, "score"] = (
            data.loc[mask, "score"] - data.loc[mask, "score"].min()
        ) / (data.loc[mask, "score"].max() - data.loc[mask, "score"].min())

        # Normalize score_top_n
        data.loc[mask, "score_top_n"] = (
            data.loc[mask, "score_top_n"] - data.loc[mask, "score_top_n"].min()
        ) / (data.loc[mask, "score_top_n"].max() - data.loc[mask, "score_top_n"].min())

    return data

random_all_data = normalize_per_scale(random_all_data)
semi_random_all_data = normalize_per_scale(semi_random_all_data)


## Chronbach's Alpha

In [4]:
# function:
def compute_cronbach_alpha(data, score = "score"):
    results = []

    # loop through experiments
    for exp, exp_data in data.groupby("experiment"):
        
        # check whether this experiment has subcategories
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            # compute alpha per category (domain)
            for domain, domain_data in exp_data.groupby("category"):
                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )
                #print(df_wide)
                if df_wide.shape[1] > 1:
                    alpha, ci = pg.cronbach_alpha(df_wide)
                else:
                    alpha, ci = None, (None, None)
                
                results.append({
                    "experiment": exp,
                    "domain": domain,
                    "alpha": alpha,
                    "alpha_CI": ci
                })
        else:
            # compute alpha for the whole experiment
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )
            #print(exp, df_wide.std().describe())
            if df_wide.shape[1] > 1:
                alpha, ci = pg.cronbach_alpha(df_wide)
            else:
                alpha, ci = None, (None, None)
            
            results.append({
                "experiment": exp,
                "domain": "total",  # no subdomain
                "alpha": alpha,
                "alpha_CI": ci
            })

    # convert to DataFrame
    return(pd.DataFrame(results))



In [5]:
# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random = compute_cronbach_alpha(random_all_data)
alpha_df_semi_random = compute_cronbach_alpha(semi_random_all_data)


# ---- Cronbach's alpha per domain or directly per scale, depending on whether the scale has subdomains ----
alpha_df_random_top_n= compute_cronbach_alpha(random_all_data, score = "score_top_n")
alpha_df_semi_random_top_n = compute_cronbach_alpha(semi_random_all_data, score = "score_top_n")



# Merge only selected columns
alpha_df = alpha_df_random.merge(
    alpha_df_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(alpha_df)

,experiment,domain,alpha_random,alpha_CI_random,alpha_semi_random,alpha_CI_semi_random
0,AUDIT scale,total,-0.071954,"[-0.597, 0.335]",-0.071954,"[-0.597, 0.335]"
1,BARRAT scale,BISa,0.102869,"[-0.35, 0.447]",0.102869,"[-0.35, 0.447]"
2,BARRAT scale,BISm,0.006240,"[-0.48, 0.383]",0.006240,"[-0.48, 0.383]"
3,BARRAT scale,BISn,-0.253252,"[-0.867, 0.222]",-0.253252,"[-0.867, 0.222]"
4,BART task,total,0.189144,"[-0.198, 0.498]",0.189144,"[-0.198, 0.498]"
5,CARE scale,CAREa,-0.024930,"[-0.536, 0.366]",-0.024930,"[-0.536, 0.366]"
6,CARE scale,CAREs,0.249995,"[-0.144, 0.542]",0.249995,"[-0.144, 0.542]"
7,CARE scale,CAREw,-0.113835,"[-0.751, 0.332]",-0.113835,"[-0.751, 0.332]"
8,CCT task,total,-0.350410,"[-1.039, 0.195]",-0.350410,"[-1.039, 0.195]"
9,DAST scale,total,0.119812,"[-0.296, 0.45]",0.119812,"[-0.296, 0.45]"


## Split-half Reliability

In [6]:
def split_half_reliability(df_items, n_splits=100):
    """Compute average split-half reliability (Spearman-Brown corrected)."""

    k = df_items.shape[1]
    
    if k < 2:
        return None, None

    cols = df_items.columns
    results = []

    for _ in range(n_splits):
        shuffled = np.random.permutation(cols)
        half1 = shuffled[:k//2]
        half2 = shuffled[k//2:]

        s1 = df_items[half1].sum(axis=1)
        s2 = df_items[half2].sum(axis=1)

        r, _ = pearsonr(s1, s2)

        if np.isfinite(r):
            r_sb = (2 * r) / (1 + r)  # Spearman-Brown correction
            results.append(r_sb)

    if len(results) == 0:
        return None, None
    
    return np.mean(results), np.std(results)


In [7]:
def compute_split_half_reliability(data, score="score"):
    rows = []  # final list of results

    for exp, exp_data in data.groupby("experiment"):

        # case 1: experiments with domains
        if exp_data["category"].notna().any() and exp != "SOEP scale":
            for domain, domain_data in exp_data.groupby("category"):

                df_wide = domain_data.pivot_table(
                    index="model", columns="item", values=score
                )

                mean_rsb, sd_rsb = split_half_reliability(df_wide)

                rows.append({
                    "experiment": exp,
                    "domain": domain,
                    "split_half_mean": mean_rsb,
                    "split_half_sd": sd_rsb
                })

        # case 2: experiments without domains
        else:
            df_wide = exp_data.pivot_table(
                index="model", columns="item", values=score
            )

            mean_rsb, sd_rsb = split_half_reliability(df_wide)

            rows.append({
                "experiment": exp,
                "domain": "total",
                "split_half_mean": mean_rsb,
                "split_half_sd": sd_rsb
            })

    return pd.DataFrame(rows)


In [8]:
split_half_reliability_random = compute_split_half_reliability(random_all_data)
split_half_reliability_semi_random = compute_split_half_reliability(semi_random_all_data)


# Merge 
split_half_rel_df = split_half_reliability_random.merge(
    split_half_reliability_semi_random,
    on=["experiment", "domain"],
    how="inner",
    suffixes=("_random", "_semi_random")
)

display(split_half_rel_df)

,experiment,domain,split_half_mean_random,split_half_sd_random,split_half_mean_semi_random,split_half_sd_semi_random
0,AUDIT scale,total,-0.065019,0.200005,-0.069942,0.184067
1,BARRAT scale,BISa,0.087750,0.184429,0.125238,0.211289
2,BARRAT scale,BISm,0.018805,0.175505,0.015085,0.175345
3,BARRAT scale,BISn,-0.239880,0.185590,-0.240392,0.181253
4,BART task,total,0.211798,0.156168,0.197471,0.157998
5,CARE scale,CAREa,-0.031277,0.213295,-0.025545,0.213709
6,CARE scale,CAREs,0.384326,0.051659,0.395205,0.046910
7,CARE scale,CAREw,-0.144509,0.177332,-0.171993,0.170892
8,CCT task,total,-0.389208,0.339602,-0.434438,0.362023
9,DAST scale,total,0.112243,0.150087,0.151583,0.174757
